<a href="https://colab.research.google.com/github/MarijanMarkovic/Numerical-modelling-of-differential-equations/blob/main/Stokes_vj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install -y --no-install-recommends fenics
!rm -rf 

!pip3 install git+https://bitbucket.org/mattiapenati/cbc.block.git #for the block matrices

In [ ]:
#!/usr/bin/env python

from fenics import *
from block import *
from block.iterative import LGMRES
from block.algebraic.petsc import AMG, LumpedInvDiag
	

# 1. mesh generation
nx, ny = 40, 40
mesh = UnitSquareMesh(nx, ny, 'crossed')

# 2. finite element space
degree = 1
V = VectorFunctionSpace(mesh, 'CG', degree+1)
Q = FunctionSpace(mesh, 'CG', degree)

# 3. problem definition
u = TrialFunction(V)
v = TestFunction(V)

p = TrialFunction(Q)
q = TestFunction(Q)


def top_boundary(x, on_boundary):
    return on_boundary and near(x[1], 1)

def other_boundary(x, on_boundary):
    return on_boundary and not near(x[1], 1)

bcs = [
    DirichletBC(V, Constant((1, 0)), top_boundary),
    DirichletBC(V, Constant((0, 0)), other_boundary)
]

a = inner(grad(u), grad(v)) * dx
bT = - div(v) * p * dx
b = -div(u) * q * dx
c = p * q * dx
L1 = dot(Constant((0, 0)), v) * dx
L2 = Constant(0) * q * dx

# 4. solution
A, f1 = assemble_system(a, L1)
BT = assemble(bT)
for bc in bcs:
    bc.apply(A, f1)
    bc.zero(BT)

B = assemble(b)
f2 = assemble(L2)

Sigma = block_mat([[A, BT], [B, 0]])
bb = block_vec([f1, f2])

M = assemble(c)

Ap = AMG(A)
Mp = LumpedInvDiag(M)

P = block_mat([[Ap, 0],
               [0, Mp]])

#solver = MinRes(Sigma)
solver = LGMRES(Sigma, precond=P)
xx = solver * bb

u = Function(V, xx[0])
p = Function(Q, xx[1])

u.rename('velocity', 'velocity')
p.rename('pressure', 'pressure')

File('velocity.pvd') << u
File('pressure.pvd') << p


Calling FFC just-in-time (JIT) compiler, this may take some time.
LGMRES converged [iter= 2, time=1.60s, res=4.3e-09]


In [ ]:
from fenics import *
from block import *
from block.iterative import LGMRES
from block.algebraic.petsc import LumpedInvDiag, LU


# 1. mesh generation
nx, ny = 40, 40
mesh = UnitSquareMesh(nx, ny, 'crossed')

# 2. finite element space
degree = 1
V = VectorFunctionSpace(mesh, 'CG', degree+1)
Q = FunctionSpace(mesh, 'CG', degree)

# 3. problem definition
u = TrialFunction(V)
v = TestFunction(V)

p = TrialFunction(Q)
q = TestFunction(Q)


def top_boundary(x, on_boundary):
    return on_boundary and near(x[1], 1)

def other_boundary(x, on_boundary):
    return on_boundary and not near(x[1], 1)

bcs = [
    DirichletBC(V, Constant((1, 0)), top_boundary),
    DirichletBC(V, Constant((0, 0)), other_boundary)
]

a = inner(grad(u), grad(v)) * dx
bT = - div(v) * p * dx
b = -div(u) * q * dx
c = p * q * dx
L1 = dot(Constant((0, 0)), v) * dx

# 4. solution
A, f1 = assemble_system(a, L1)
BT = assemble(bT)
for bc in bcs:
    bc.apply(A, f1)
    bc.zero(BT)

B = assemble(b)

M = assemble(c)
Mp = LumpedInvDiag(M)

Ainv = LU(A)
S = B * Ainv * BT

b = B * Ainv * f1
#solver = LGMRES(S)
solver = LGMRES(S, precond=Mp)
x1 = solver * b

x0 = Ainv * (f1 - BT * x1)

u = Function(V, x0)
p = Function(Q, x1)

u.rename('velocity', 'velocity')
p.rename('pressure', 'pressure')

File('velocity.pvd') << u
File('pressure.pvd') << p

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
LGMRES converged [iter= 1, time=0.31s, res=2.0e-08]
